In [1]:
import numpy as np
import pytesseract
from fuzzywuzzy import fuzz
from PIL import Image, ImageDraw, ImageOps
from PIL import ImageFont
import argparse

#you can simply comment all of the next 4 lines and un comment the 3 lines in main if you want to test on jupyter
parser = argparse.ArgumentParser(description = 'create a deblurred copy of each image in a folder and store them in a specified folder ')
parser.add_argument('input_folder', type= str , help = 'input folder path')
parser.add_argument('output_folder', type= str , help = 'output folder path') 
args = parser.parse_args()

pytesseract.pytesseract.tesseract_cmd = r'C:\Users\hp\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
def normalize(img):
    """
    Linear histogram normalization
    """
    arr = np.array(img, dtype=float)

    arr = (arr - arr.min()) * (255 / arr[:, :50].min())
    arr[arr > 255] = 255
    arr[arr < 0] = 0

    return Image.fromarray(arr.astype('uint8'), 'L')


def evaluateImage(imageFile, trueTextFile):
    with open(trueTextFile, 'r') as f:
        trueText = f.readlines()

    # remome \n character
    trueText = [text.rstrip() for text in trueText]

    # load image and convert to grayscale
    img = Image.open(imageFile)
    # img.show()
    img = ImageOps.grayscale(img)
    # img.show()
    w, h = img.size

    img = normalize(img)
    # img.show()

    # resize image to improve OCR
    img = img.resize((int(w / 2), int(h / 2)))
    w, h = img.size

    # run OCR
    options = r'--oem 1 --psm 6 -c load_system_dawg=false -c load_freq_dawg=false  -c textord_old_xheight=0  -c textord_min_xheight=100 -c ' \
              r'preserve_interword_spaces=0'
    OCRtext = pytesseract.image_to_string(img, config=options)

    # debug mode
    showOCRImage = False
    if showOCRImage:
        boxes = pytesseract.image_to_boxes(img, config=options)
        draw = ImageDraw.Draw(img)

        font = ImageFont.truetype('./verdanaRef.ttf', 30)
        for b in boxes.splitlines():
            text = b[0]
            b = [int(x) for x in b.split(' ')[1:]]
            draw.rectangle([b[0], h - b[1], b[2], h - b[3]], fill=None, outline='#ff0000', width=1)
            draw.text((b[0], h - b[1]), text, font=font, fill=0)

        img.show()

    # removes form feed character  \f
    OCRtext = OCRtext.replace('\n\f', '').replace('\n\n', '\n')

    # split lines
    OCRtext = OCRtext.split('\n')

    # remove empty lines
    OCRtext = [x.strip() for x in OCRtext if x.strip()]

    # check if OCR extracted 3 lines of text
    print('File:' + imageFile)
    print('True text (middle line): %s' % trueText[1])

    if len(OCRtext) != 3:
        print('ERROR: OCR text does not have 3 lines of text!')
        print(OCRtext)
        return None
    else:
        score = fuzz.ratio(trueText[1], OCRtext[1])
        print('OCR  text (middle line): %s' % OCRtext[1])
        print('Score: %d' % score)

        return float(score)
    
    
def main(input_folder , output_folder):
    print("input folder is : " + input_folder)
    print("outputfolder is : " + output_folder)

if __name__ == "__main__":

    #InputFile = './example.png'
    #trueText = './example_TrueText.txt'

    #score = evaluateImage(InputFile, trueText)
    main(args.input_folder , args.output_folder)


C:\Users\hp\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
usage: ipykernel_launcher.py [-h] input_folder output_folder
ipykernel_launcher.py: error: the following arguments are required: output_folder


SystemExit: 2

C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
